<a href="https://colab.research.google.com/github/aliebi/language-modeling/blob/master/LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [0]:
from fastai import *
from fastai.text import *
import fastai.utils.collect_env

In [0]:
# bs=48
# bs=24
bs=192

In [0]:
torch.cuda.set_device(0)

read full dataset of imdb

In [5]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/root/.fastai/data/imdb/unsup'),
 PosixPath('/root/.fastai/data/imdb/tmp_lm'),
 PosixPath('/root/.fastai/data/imdb/train'),
 PosixPath('/root/.fastai/data/imdb/tmp_clas'),
 PosixPath('/root/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/root/.fastai/data/imdb/test'),
 PosixPath('/root/.fastai/data/imdb/README')]

In [0]:

data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1, seed=42)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs, num_workers=1))

In [7]:
len(data_lm.vocab.itos),len(data_lm.train_ds)

(60000, 90000)

In [8]:
data_lm.show_batch()

idx,text
0,"of the time i find her shtick a little tedious , and where i really like her as a supporting player , when she 's the lead i start to get a headache . xxmaj perhaps that 's just me , but if you ever thought she may be a little over the top , this is not the film for you . \n \n xxmaj paul xxmaj rudd"
1,"transformation from troubled people to those xxunk having been made to accept being xxunk who have lost everything they have and thereof can do anything , regardless of any value and concept associated with humanity such as love , affection , empathy , honor , self - respect . xxmaj the psychos in the movie had n't committed any crime till the story , none was recorded a least ."
2,"of the titles for this release ) although the pictures sometimes show nothing more than xxmaj ornella xxmaj muti naked in a photo - me - booth . xxmaj they end up in the mansion of a rich lady ( xxmaj irene xxmaj pappas giving a performance less lively than some of the furniture and obviously wishing she were elsewhere ) because they run out of gasoline , and when"
3,"be different if something happened at all in the film ... it does not . xxmaj just try to think of xxmaj curse of the xxmaj screaming xxmaj dead / xxmaj curse of the xxmaj cannibal xxmaj confederates xxup without xxup any xxup zombies . xxmaj yes , i am serious . xxmaj that is just what this film is . \n \n xxmaj honestly ... this is the"
4,"is just seriously a bad time . xxmaj the first few minutes are good enough , with a good film score . xxmaj from the start , you begin to sense that this is an atmospheric film that may lose you in its boredom sometimes but still maintain enough of an edge to hold you for the xxunk hour running length . xxmaj fulci gets a lot of credit for"


In [0]:
data_lm.save('lm_databunch')


In [0]:
data_lm = load_data(path, 'lm_databunch', bs=bs)

here we make LM for our imdb dataset that 

In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [0]:
wiki_itos = pickle.load(open(Config().model_path()/'wt103-fwd/itos_wt103.pkl', 'rb'))

In [13]:
len(wiki_itos)

60000

In [0]:
wiki_itos

In [15]:

vocab = data_lm.vocab
vocab.stoi['green']

1377

In [16]:

vocab.itos[vocab.stoi["green"]]

'green'

In [0]:
awd = learn_lm.model[0]

# Difference in vocabulary between IMDB and Wikipedia
 It is to be expected that the two sets have some different vocabulary words

In [22]:
len(wiki_itos),len(vocab.itos)

(60000, 60000)

In [0]:

i, unks = 0, []
while len(unks) < 50:
    if data_lm.vocab.itos[i] not in wiki_itos: unks.append((i,data_lm.vocab.itos[i]))
    i += 1
    
    
wiki_words = set(wiki_itos)

imdb_words = set(vocab.itos)

wiki_not_imbdb = wiki_words.difference(imdb_words)

imdb_not_wiki = imdb_words.difference(wiki_words)




wiki_not_imdb_list = []

for i in range(100):
    word = wiki_not_imbdb.pop()
    wiki_not_imdb_list.append(word)
    wiki_not_imbdb.add(word)

In [25]:
wiki_not_imdb_list[:15]

['bulliard',
 'ung',
 'polyps',
 '1035',
 'rodrigues',
 'groupe',
 'bajrang',
 'jackalope',
 'harclay',
 'barbossa',
 'achtung',
 'glazing',
 'retook',
 'thistle',
 'speciation']

In [0]:
imdb_not_wiki_list = []

for i in range(100):
    word = imdb_not_wiki.pop()
    imdb_not_wiki_list.append(word)
    imdb_not_wiki.add(word)

In [27]:
imdb_not_wiki_list[:15]

['desecrate',
 'desny',
 'encolpio',
 'fragata',
 'curdling',
 'popstar',
 'senar',
 'yuelin',
 'reawaken',
 'tp',
 'markov',
 'chore',
 'hadou',
 'blackmer',
 'gentleness']

In [28]:
"desecrate" in wiki_words, "desecrate" in imdb_words

(False, True)

# _______________________________________make embedding

now we should encode the LM of IMDB :

In [0]:
enc = learn_lm.model[0].encoder

In [19]:
enc


Embedding(60000, 400, padding_idx=1)

as you see above we make Embedding of word 

# Generating fake movie reviews (using wiki-text model)

In [0]:
TEXT = "The color of the sky is"
N_WORDS = 40
N_SENTENCES = 2

<!--  --> temperature is for randomized and you decide how much words can be random.

In [32]:

print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

The color of the sky is a natural significance , for instance in the Summer of Love , the Tibetan Dream ( Buddhist epics ) , and in the Buddhist goddess Torah ( The Tree of
The color of the sky is dark , dark , and dark , and the dark red on the sky is considered to be painted by some people . In the 1970s , Frank Befriended and the American team were looking for


In [0]:

TEXT = "my idea about this movie "
N_WORDS = 30
N_SENTENCES = 2

In [39]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.10) for _ in range(N_SENTENCES)))


my idea about this movie  = 
 
  The idea of a film adaptation of the novel was first proposed by American novelist William Shakespeare in the early 19th century .
my idea about this movie  = 
 
  The idea of a film adaptation of the novel was first proposed by American author Robert Oppenheimer in the 1950s . He wrote


In [40]:
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.70) for _ in range(N_SENTENCES)))

my idea about this movie  = 
 
  The history of a cinema is a matter of debate . It is a science - fiction film and is a set of several stories .
my idea about this movie  = 
 
  The philosophy of architecture is a theory of how the modern universe begins to expand in the Universe and form a new universe . It


as you see above temperature has affect on randomize of sentences ,in the first part two sentences are same but in second part sentences are different 


# more example :

In [0]:

N_WORDS = 40
N_SENTENCES = 5


In [46]:
TEXT = "i liked this movie because"
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

i liked this movie because of its violence . However , the film did not contain any religious imagery . It was a request from Christianity for a commercial film snipped , and this was later confirmed by the Olympic
i liked this movie because of the film 's success . The film 's script was written by French composer Jean Bernard Clemens and was sung by Pauline Kael . The film was met with mixed reception
i liked this movie because it " got a good idea " of the film . Nevertheless , it was one of the most popular film roles of the 1960s , with Gary Cooper 's portrayal as a slave as follows :
i liked this movie because of America 's own culture of the 1950s , and the great deal between it and the American public to make it an international benefits . Although many people did not see it , it was a
i liked this movie because it was too massive and silly and that it was not for anyone . As a result , the film was virtually forgotten . The film was released in Australia , South Africa and


In [47]:
TEXT = "I hated this movie"
print("\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

I hated this movie hell = 
 
  i ' m not a teacher or a fool , but it is a whole . It 's very difficult to imagine the movie is really real . It just goes that original piece of
I hated this movie as " an AMC show " . The British television weekend network BBC One , which had a high - rated television series , went on to produce a free - to - air show
I hated this movie and wrote it as a personal wonder . " It was the first time i really did it , " he said . " It 's just a little more than a tragedy , because it 's a
I hated this movie in the United States , starring Tom Cruise in The Godfather ( 2004 ) . The Charleston Daily News reported that the film was used to watch the non -
I hated this movie in the United States as it did in Washington , D.C. , in the United States . Instead , it focused on the American West , where the role of the
